# Purpose
A recent report (https://fuelcycleoptions.inl.gov/SiteAssets/SitePages/Home/Evaluation%20of%20NPP%20and%20CPP%20Sites%20Aug%2016%202024.pdf) identified locations of potential new large LWR builds on existing plant sites. In this notebook, we will demonstrate a programmatic method to add these plants for analysis.

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib
from collections import defaultdict
matplotlib.use
import matplotlib.pyplot as plt
#plt.style.use('../plotting.mplstyle')

import textwrap
import xml.etree.ElementTree as ET

In [2]:
sys.path.insert(0,'../../../scripts')

In [4]:
def generate_est_facility_xml(df, output_dir):
    """
    Generate the XML string for the estimated facilities from the given dataframe.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the information about the reactors.
    output_dir : str
        The directory to output the XML files to.

    Returns
    -------
    str
        The XML string for the reactor facility.

    Notes
    -----
    * This function assumes that LWRs will all operate for 80 years unless they are prematurely retired.
    """

    # if the output directory does not exist, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for state in range(len(df['Plant state'])):
        st = df.loc[state,'Plant state']
        for number in range(df.loc[state,'Estimated sites with space for additional 1117 MWe reactor']):
            xml_string_large = textwrap.dedent(f"""
        <facility>
        <name>{st}_{number}_large_est</name>
        <lifetime>960</lifetime>
        <config>
            <Reactor>
            <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>
            <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>
            <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>
            <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>
            <cycle_time>18</cycle_time>
            <refuel_time>1</refuel_time>
            <assem_size>427.38589211618256</assem_size>
            <n_assem_core>193</n_assem_core>
            <n_assem_batch>80</n_assem_batch>
            <power_cap>1117</power_cap>
            </Reactor>
        </config>
        </facility>
        """).strip()
            with open(f'{output_dir}{st}_{number}_large_est.xml', 'w') as f:
                f.write(xml_string_large)

        for number in range(df.loc[state,'Estimated sites with space for additional 600 MWe reactor']):
            xml_string_small = textwrap.dedent(f"""
        <facility>
        <name>{st}_{number}_large_est</name>
        <lifetime>960</lifetime>
        <config>
            <Reactor>
            <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>
            <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>
            <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>
            <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>
            <cycle_time>18</cycle_time>
            <refuel_time>1</refuel_time>
            <assem_size>427.38589211618256</assem_size>
            <n_assem_core>193</n_assem_core>
            <n_assem_batch>80</n_assem_batch>
            <power_cap>600</power_cap>
            </Reactor>
        </config>
        </facility>
        """).strip()
            with open(f'{output_dir}{st}_{number}_small_est.xml', 'w') as f:
                f.write(xml_string_small)

    return print('Successfully generated reactor XML files.')

In [5]:
# I pulled these numbers from Table 7 from the link at the top
currently_operating_data = {
    'Plant state': ['AL', 'AR', 'AZ', 'CA', 'CT', 'FL', 'GA', 'IL', 'KS', 'LA', 'MD', 'MI', 'MN', 'MO', 'MS', 'NC', 'NE', 'NH', 'NJ', 'NY', 'OH', 'PA', 'SC', 'TN', 'TX', 'VA', 'WA', 'WI'],
    'Number of sites': [2, 1, 1, 1, 1, 2, 2, 6, 1, 2, 1, 2, 2, 1, 1, 3, 1, 1, 2, 3, 2, 4, 4, 2, 2, 2, 1, 1],
    'Number of units': [5, 2, 3, 2, 2, 4, 6, 11, 1, 2, 2, 3, 3, 1, 1, 5, 1, 1, 3, 4, 2, 8, 7, 4, 4, 4, 1, 2],
    'Sites with planned additional reactor': [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 1, 0],
    'Sites that considered a COL *': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0],
    'Estimated sites with space for additional 1117 MWe reactor': [2, 0, 0, 0, 0, 0, 2, 4, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1],
    'Estimated sites with space for additional 600 MWe reactor': [2, 1, 1, 0, 0, 2, 2, 6, 1, 2, 1, 2, 1, 1, 1, 2, 1, 0, 0, 2, 2, 2, 2, 1, 2, 2, 1, 1]
}

currently_operating_df = pd.DataFrame(currently_operating_data)
currently_operating_df

,Plant state,Number of sites,Number of units,Sites with planned additional reactor,Sites that considered a COL *,Estimated sites with space for additional 1117 MWe reactor,Estimated sites with space for additional 600 MWe reactor
0,AL,2,5,0,0,2,2
1,AR,1,2,0,0,0,1
2,AZ,1,3,1,0,0,1
3,CA,1,2,0,0,0,0
4,CT,1,2,0,0,0,0
5,FL,2,4,0,1,0,2
6,GA,2,6,0,0,2,2
7,IL,6,11,1,0,4,6
8,KS,1,1,0,0,1,1
9,LA,2,2,1,0,1,2


In [6]:
generate_est_facility_xml(currently_operating_df,'../reactors/lwrs_est/')

Successfully generated reactor XML files.


The only thing left to do is to decide when these reactors will be deployed, as that is not part of the work these files are generated from.